<h1>Prophet</h1>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from fbprophet import Prophet
from sklearn.metrics import mean_squared_error
from numba import jit
import math

In [2]:
train_1 = pd.read_csv('../Data/train_1.csv')
train_1.head()

,Page,2015-07-01,2015-07-02,2015-07-03,2015-07-04,2015-07-05,2015-07-06,2015-07-07,2015-07-08,2015-07-09,...,2016-12-22,2016-12-23,2016-12-24,2016-12-25,2016-12-26,2016-12-27,2016-12-28,2016-12-29,2016-12-30,2016-12-31
0,2NE1_zh.wikipedia.org_all-access_spider,18.0,11.0,5.0,13.0,14.0,9.0,9.0,22.0,26.0,...,32.0,63.0,15.0,26.0,14.0,20.0,22.0,19.0,18.0,20.0
1,2PM_zh.wikipedia.org_all-access_spider,11.0,14.0,15.0,18.0,11.0,13.0,22.0,11.0,10.0,...,17.0,42.0,28.0,15.0,9.0,30.0,52.0,45.0,26.0,20.0
2,3C_zh.wikipedia.org_all-access_spider,1.0,0.0,1.0,1.0,0.0,4.0,0.0,3.0,4.0,...,3.0,1.0,1.0,7.0,4.0,4.0,6.0,3.0,4.0,17.0
3,4minute_zh.wikipedia.org_all-access_spider,35.0,13.0,10.0,94.0,4.0,26.0,14.0,9.0,11.0,...,32.0,10.0,26.0,27.0,16.0,11.0,17.0,19.0,10.0,11.0
4,52_Hz_I_Love_You_zh.wikipedia.org_all-access_s...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,48.0,9.0,25.0,13.0,3.0,11.0,27.0,13.0,36.0,10.0


In [3]:
train_1_reduce = train_1[0:100]
train_1_reduce = train_1_reduce.fillna(0)
train_1_reduce = train_1_reduce.set_index('Page')
train_1_reduce = train_1_reduce.T.rename_axis('Dates')
train_1_reduce.head()

Page,2NE1_zh.wikipedia.org_all-access_spider,2PM_zh.wikipedia.org_all-access_spider,3C_zh.wikipedia.org_all-access_spider,4minute_zh.wikipedia.org_all-access_spider,52_Hz_I_Love_You_zh.wikipedia.org_all-access_spider,5566_zh.wikipedia.org_all-access_spider,91Days_zh.wikipedia.org_all-access_spider,A'N'D_zh.wikipedia.org_all-access_spider,AKB48_zh.wikipedia.org_all-access_spider,ASCII_zh.wikipedia.org_all-access_spider,...,Lovelyz_zh.wikipedia.org_all-access_spider,Lulu_zh.wikipedia.org_all-access_spider,MADTOWN_zh.wikipedia.org_all-access_spider,MAMAMOO_zh.wikipedia.org_all-access_spider,MONSTA_X_zh.wikipedia.org_all-access_spider,Madame_Antoine_zh.wikipedia.org_all-access_spider,Mamamoo_zh.wikipedia.org_all-access_spider,Mike_D._Angelo_zh.wikipedia.org_all-access_spider,Miss_A_zh.wikipedia.org_all-access_spider,Missing9_zh.wikipedia.org_all-access_spider
Dates,,,,,,,,,,,,,,,,,,,,,
2015-07-01,18.0,11.0,1.0,35.0,0.0,12.0,0.0,118.0,5.0,6.0,...,2.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,3.0,0.0
2015-07-02,11.0,14.0,0.0,13.0,0.0,7.0,0.0,26.0,23.0,3.0,...,6.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,9.0,0.0
2015-07-03,5.0,15.0,1.0,10.0,0.0,4.0,0.0,30.0,14.0,5.0,...,5.0,0.0,1.0,17.0,0.0,0.0,0.0,0.0,12.0,0.0
2015-07-04,13.0,18.0,1.0,94.0,0.0,5.0,0.0,24.0,12.0,12.0,...,21.0,0.0,0.0,11.0,0.0,0.0,0.0,0.0,14.0,0.0
2015-07-05,14.0,11.0,0.0,4.0,0.0,20.0,0.0,29.0,9.0,6.0,...,4.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,6.0,0.0


In [4]:
def smape_fast(y_true, y_pred):
    out = 0
    for i in range(y_true.shape[0]):
        a = y_true[i]
        b = y_pred[i]
        c = a+b
        if c == 0:
            continue
        out += math.fabs(a - b) / c
    out *= (200.0 / y_true.shape[0])
    return out

In [10]:
metrics_df = pd.DataFrame(columns=['RMSE','SMAPE','MAPE'])

for column in train_1_reduce.columns:
    print(column)
    df = pd.DataFrame(train_1_reduce[column])
    df = df.reset_index()
    df.columns = ['ds', 'y']
    dftrain = df[0:500]
    dftest = df[500:550]
    m = Prophet(yearly_seasonality = False)
    #m.add_seasonality(name='monthly', period=30.5, fourier_order=5)
    m.add_seasonality(name='quarterly', period=90, fourier_order=5)
    m.fit(dftrain)
    future = m.make_future_dataframe(periods=50)
    future.tail()
    forecast = m.predict(future)
    forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
    predictions = forecast['yhat'][500:550]
    rmse = np.sqrt(mean_squared_error(dftest['y'], predictions))
    smape = smape_fast(dftest['y'].values, predictions.values)
    mape = np.mean(np.abs(predictions.values - dftest['y'].values)/np.abs(dftest['y'].values))
    print("Test RMSE: %.3f" % rmse)
    print("Test SMAPE: %.3f" % smape)
    print("Test MAPE: %.3f" % mape)
    metrics_df.loc[column] = [rmse,smape,mape]

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


2NE1_zh.wikipedia.org_all-access_spider


INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Test RMSE: 30.039
Test SMAPE: 45.023
Test MAPE: 0.469
2PM_zh.wikipedia.org_all-access_spider


INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


...

Test RMSE: 56.526
Test SMAPE: 57.431
Test MAPE: 0.720
Mike_D._Angelo_zh.wikipedia.org_all-access_spider


INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Test RMSE: 7.512
Test SMAPE: 63.222
Test MAPE: 0.610
Miss_A_zh.wikipedia.org_all-access_spider


INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Test RMSE: 5.871
Test SMAPE: 31.658
Test MAPE: 0.379
Missing9_zh.wikipedia.org_all-access_spider
Test RMSE: 21.319
Test SMAPE: 75.511
Test MAPE: 1.288


In [6]:
metrics_df

,RMSE,SMAPE,MAPE
2NE1_zh.wikipedia.org_all-access_spider,30.735477,65.861115,1.083894
2PM_zh.wikipedia.org_all-access_spider,85.139280,48.532741,0.589273
3C_zh.wikipedia.org_all-access_spider,29.162703,59.860367,inf
4minute_zh.wikipedia.org_all-access_spider,8.434781,38.380748,0.445983
52_Hz_I_Love_You_zh.wikipedia.org_all-access_spider,26.795707,50.635993,0.421593
...,...,...,...
Madame_Antoine_zh.wikipedia.org_all-access_spider,6.202532,73.461402,1.759797
Mamamoo_zh.wikipedia.org_all-access_spider,58.074119,57.681858,0.581778
Mike_D._Angelo_zh.wikipedia.org_all-access_spider,21.654388,141.414727,2.482618
Miss_A_zh.wikipedia.org_all-access_spider,6.514576,34.958909,0.382283


In [11]:
moyenne_RMSE = metrics_df['RMSE'].mean()
somme_RMSE = metrics_df['RMSE'].sum()
moyenne_SMAPE = metrics_df['SMAPE'].mean()
moyenne_MAPE = metrics_df['MAPE'].mean()

In [8]:
metrics_df['MAPE']

2NE1_zh.wikipedia.org_all-access_spider                1.083894
2PM_zh.wikipedia.org_all-access_spider                 0.589273
3C_zh.wikipedia.org_all-access_spider                       inf
4minute_zh.wikipedia.org_all-access_spider             0.445983
52_Hz_I_Love_You_zh.wikipedia.org_all-access_spider    0.421593
                                                         ...   
Madame_Antoine_zh.wikipedia.org_all-access_spider      1.759797
Mamamoo_zh.wikipedia.org_all-access_spider             0.581778
Mike_D._Angelo_zh.wikipedia.org_all-access_spider      2.482618
Miss_A_zh.wikipedia.org_all-access_spider              0.382283
Missing9_zh.wikipedia.org_all-access_spider            1.300898
Name: MAPE, Length: 100, dtype: float64

In [24]:
#Pas de saisonalité ajoutée
print("Test moyenne RMSE: %.3f" % moyenne_RMSE)
print("Test somme RMSE: %.3f" % somme_RMSE)
print("Test moyenne SMAPE: %.3f" % moyenne_SMAPE)
print("Test moyenne MAPE: %.3f" % moyenne_MAPE)

Test moyenne RMSE: 20.977
Test somme RMSE: 2097.731
Test moyenne SMAPE: 36.670
Test moyenne MAPE: inf


In [19]:
#Saisonalité mensuelle
print("Test moyenne RMSE: %.3f" % moyenne_RMSE)
print("Test somme RMSE: %.3f" % somme_RMSE)
print("Test moyenne SMAPE: %.3f" % moyenne_SMAPE)
print("Test moyenne MAPE: %.3f" % moyenne_MAPE)

Test moyenne RMSE: 21.452
Test somme RMSE: 2145.172
Test moyenne SMAPE: 77.344
Test moyenne MAPE: inf


In [12]:
#Saisonalité trimestrielle
ignore_index=True

Test moyenne RMSE: 22.351
Test somme RMSE: 2235.063
Test moyenne SMAPE: 23.050
Test moyenne MAPE: inf


In [9]:
#Saisonalité annuelle
print("Test moyenne RMSE: %.3f" % moyenne_RMSE)
print("Test somme RMSE: %.3f" % somme_RMSE)
print("Test moyenne SMAPE: %.3f" % moyenne_SMAPE)
print("Test moyenne MAPE: %.3f" % moyenne_MAPE)

Test moyenne RMSE: 22.676
Test somme RMSE: 2267.615
Test moyenne SMAPE: 57.243
Test moyenne MAPE: inf
